We start by importing relevant packages.

In [115]:
import json
import numpy as np
from scipy.interpolate import interp1d
import plotly.graph_objects as go

Let's load the data.

In [116]:
# Specify the path to the JSON file
filepath = "data/data.json"

# Open the JSON file and load its contents into a Python dictionary
with open(filepath, "r") as file:
    data = json.load(file)

I want the keys to be integers.

In [117]:
data = {int(key): value for key, value in data.items()}

We also have to handle the fact that some of the scenarios were renamed.

In [118]:
# Key mapping for normalization
key_mapping = {
    "Reference": "Current Policies",
    "New Policies": "Stated Policies",
}

relevant_keys = ["Current Policies", "Stated Policies", "Announced Pledges"]


# Step 0: Normalize keys across all values
def normalize_keys(data_dict):
    for content in data_dict.values():
        values = content.get("values", {})
        normalized_values = {}
        for key, value in values.items():
            normalized_key = key_mapping.get(key, key)  # Map key if in key_mapping
            if normalized_key in normalized_values:
                print(
                    f"Warning: Duplicate normalized key '{normalized_key}'. Overwriting previous entry."
                )
            normalized_values[normalized_key] = value
        content["values"] = normalized_values

In [119]:
normalize_keys(data)

I want to be able to interpolate the data.

In [120]:
def generate_splines(data_dict):
    for _, content in data_dict.items():
        # Extract years and values
        year_values = content["year"]
        value_dict = content["values"]

        # Create a new "splines" dictionary
        splines = {}
        for key, values in value_dict.items():
            # Handle null values by filtering them out
            valid_data = [
                (yr, val) for yr, val in zip(year_values, values) if val is not None
            ]
            if len(valid_data) >= 2:  # Ensure there are enough points to interpolate
                valid_years, valid_values = zip(*valid_data)
                spline = interp1d(
                    valid_years, valid_values, kind="cubic", fill_value="extrapolate"
                )
                splines[key] = spline  # Store the spline function
            else:
                splines[key] = None  # Not enough data points or missing values

        # Add "splines" to the content
        content["splines"] = splines

In [121]:
generate_splines(data)

Now I need a method of extracting the predictions for the same scenario over the years.

In [122]:
def obtain_values(data_dict, unique_keys, year_test):
    values = {key: [] for key in unique_keys}
    years = []  # To keep track of x-axis years

    for year, content in data_dict.items():
        years.append(year)
        splines = content.get("splines", {})
        for key in unique_keys:
            if key in splines and splines[key] is not None:  # Valid spline exists
                values[key].append(splines[key](year_test).item())
            else:
                values[key].append(None)  # Append None if no valid spline

    return years, values

In [123]:
reference_year = 2019

years, values_2019 = obtain_values(data, relevant_keys, reference_year)